
<table>
  <tr>
    <td> <center><img src="images/header1.png" width=400></center> </td>
  </tr>
</table>

<h1><center>Основы машинного обучения</center></h1>
<hr>
<h2><center>Методы обучения без учителя: Кластеризация (практика)</center></h2>
<h3><center>Ефимов Владислав</center></h3>

In [1]:
%matplotlib inline

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-talk')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

C:\Users\vald_\AppData\Local\Temp\ipykernel_18920\3813181654.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-talk')


In [10]:
# Uncomment if you are using colab
# !mkdir ./data
# !wget https://raw.githubusercontent.com/vadim0912/ML2023/main/lecture06/data/diet.csv -O ./data/diet.csv
# !wget https://raw.githubusercontent.com/vadim0912/ML2023/main/lecture06/data/snsdata.csv -O ./data/snsdata.csv

# Применение K-means

Загрузите [данные](https://github.com/brenden17/sklearnlab/blob/master/facebook/snsdata.csv) в которых содержится описание интересов профилей учеников старшей школы США.

In [11]:
df_sns = pd.read_csv('data/snsdata.csv', sep=',')
df_sns.head()

,gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
0,2006,M,18.982,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2006,F,18.801,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2006,M,18.335,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2006,F,18.875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,NaN,18.995,10,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,1,1


Данные устроены так: 
* Год выпуска
* Пол
* Возраст
* Количество друзей
* 36 ключевых слов, которые встречаются в профилe facebook (интересы, сообщества, встречи)

#### Задание

* Удалите все признаки кроме 36 ключевых слов.
* Нормализуйте данные - из каждого столбца вычтите его среднее значение и поделите на стандартное отклонение.
* Используйте метод k-means чтобы выделить 9 кластеров
* Попробуйте проинтерпретировать каждый кластер проанализировав полученные центройды (Некоторые кластеры могут быть очень большие и очень маленькие - плохо интерпретируются)



# Рационы питания в странах

Загрузите набор данных о пищевом рационе в разных странах мира `diet.csv`

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

In [13]:
df = pd.read_csv('data/diet.csv', sep=';').iloc[:, :-1]

In [14]:
df.head()

,Countries,Energy (kcal/day),Protein (g/day),Fats (g/day),Carbohydrates (g/day),Animal Products + (kcal/day),Animal Fats (kcal/day),Bovine Meat (kcal/day),"Butter, Ghee (kcal/day)",Cheese (kcal/day),...,Soyabean Oil (kcal/day),Starchy Roots (kcal/day),Sugar & Sweeteners (kcal/day),Sugar (Raw Equivalent) (kcal/day),"Sugar, Raw Equivalent (kcal/day)","Sugar, Refined Equiv (kcal/day)",Vegetable Oils (kcal/day),Vegetables (kcal/day),Wheat (kcal/day),Wine (kcal/day)
0,Albania,2860.0,96,86,425.50,813.0,49,62,11,50,...,2,57,193,187,191,187,174,94,1166,6
1,Argentina,2980.0,94,100,426.00,823.0,72,342,28,90,...,43,100,406,337,405,337,311,51,914,59
2,Australia,3120.0,107,134,371.50,1033.0,124,142,62,107,...,17,87,423,407,415,407,435,67,559,39
3,Austria,3740.0,111,162,459.50,1219.0,320,59,102,193,...,89,113,437,404,424,404,442,61,617,55
4,Bangladesh,2200.0,48,25,445.75,65.0,5,5,3,0,...,48,42,59,29,59,29,131,10,180,0


In [15]:
scaler = StandardScaler()

In [16]:
X = df.iloc[:, 1:]
X = scaler.fit_transform(X)


1. Отнормируйте данные с помощью `RobustScaler` или `StandardScaler`
2. Используйте метод K-средних. Выберите число кластеров с помощью критерия силуэта
3. Найдите выборосы и проинтерпретируйте кластеры

In [17]:
## Your Code Here

## Оценка `epsilon` для DBSCAN

То что нам не нужно определять количество выходных кластеров в DBSCAN - это конечно хорошо, но как определить `min_pts` и `epsilon`?) Есть одна методика

Будем считать, что MinPts нам дан свыше (например MinPts = 2). Воспользуемся следующим способом оценки:

* Нормализуем признаки, например с помощью `RobustScaler` или `StandartScaler`
* Расчитаем расстояние до k=MinPts ближайшего соседа каждой точки (класс `NearestNeighbors` и метод `kneighbors`)
* Отсортируем полученный массив и выведите его на график
* По графику будет примерно понятно, сколько точек уйдет в шум, а сколько попадет в полноценный кластер

In [18]:
from sklearn.neighbors import NearestNeighbors

array([[-0.00321914, -0.06351547, -0.10782048, ...,  0.81434012,
         1.64193249, -0.16778847],
       [ 0.20415642, -0.06970157,  0.25378359, ..., -0.25354278,
         0.92710804,  0.1683785 ],
       [ 0.44609459, -0.02949189,  1.1319649 , ...,  0.143809  ,
        -0.07988671,  0.04152304],
       ...,
       [ 0.99909611, -0.03877105,  1.23528035, ...,  0.16864349,
         0.52431014,  0.00980917],
       [ 1.56937892, -0.00784053,  1.70019987, ...,  0.39215386,
         0.04492391, -0.12338906],
       [-1.02281569, -0.15321399, -0.67605545, ...,  0.01963657,
         1.89439033, -0.18681679]])

# Задача про кластеризацию текстов (ДЗ)

Рассмотрим коллекцию новостных сообщений за первую половину 2017 года. Про каждое новостное сообщение известны:
* его заголовок и текст
* дата его публикации
* событие, о котором это новостное сообщение написано 
* его рубрика 

In [ ]:
df = pd.read_csv('./data/news.csv', encoding='utf8')
df.head()

In [ ]:
df.loc[:, 'class'].value_counts()

Попробуем кластеризовать документы (каким-либо методом) и сравним полученное разбиение с данными рубликами с помощью ARI

### Стандартная предобработка текстов

In [ ]:
import re

# Оставляем только кириллические символы
regex = re.compile(u"[А-Яа-я]+")

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))


df.text = df.text.str.lower()
df.loc[:, 'text'] = df.text.apply(words_only)

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

# Удаляем стоп-слова
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', u'также',  'т', 'д', '-', '-']

def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return u" ".join([token for token in text.split() if not token in mystopwords])
    except:
        return u""
    
df.text = df.text.apply(remove_stopwords)   

In [ ]:
%%time 
from pymystem3 import Mystem


# нормализуем текст
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

df.text = df.text.apply(lemmatize)

In [ ]:
mystoplemmas = [u'который', u'прошлый', u'сей', u'свой', u'наш', u'мочь']

# Еще кое-что удаляем
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""

df.text = df.text.apply(remove_stoplemmas)  

In [ ]:
df.text[0]

## Вычисление сходства

С помощью `TfidfVectorizer` и `pairwise_distances` расчитайте косинусное расстояние между всеми парами документов к корпусе

Запишите результат в переменную `S`

In [ ]:
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

In [ ]:
vect = TfidfVectorizer()
texts = df.text.values

In [ ]:
# Your Code Here

In [ ]:
S = ...

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(data=S, cmap = 'Spectral').set(xticklabels=[],yticklabels=[])


Вы должны пронаблюдать, что между некоторыми текстами есть довольно выскокое сходство по содержанию слов. 

Попробуем их кластеризовать

## DBSCAN
* Воспользуйтесь методикой оценки параметров для алгоритма DBSCAN. Используйте косинусную меру близости
* Выделите кластеры. Для каждого кластера (кроме -1, если он будет) выведите несколько текстов и умозрительно определите его тематику
* Оцените сходство с изначальными рубриками визуально (с помощью матрицы перемешивания) и с помощью ARI

In [ ]:
# Your Code Here

In [ ]:
labels = ...

In [ ]:
adjusted_rand_score(true_label, labels)

In [ ]:
pd.crosstab(df.loc[:, 'class'], labels)